https://python.langchain.com/v0.1/docs/integrations/chat/azure_chat_openai/

In [ ]:
%pip install -qU langchain-openai

In [ ]:
import nest_asyncio

nest_asyncio.apply()

# If you're running this inside a notebook, you also need to patch the AsyncIO loop.

In [ ]:
import os

os.environ["AZURE_OPENAI_API_KEY"] = "***"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://***.openai.azure.com/"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-02-01"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "gpt432k"

In [ ]:

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import StrOutputParser

model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)


In [ ]:
message = HumanMessage(
    content="Translate this sentence from English to French. I love programming."
)
model.invoke([message])

In [ ]:
messages = [
    ("system", "You are a helpful assistant that translates English to French."),
    ("human", "Translate this sentence from English to French. I love programming."),
]

response = model.invoke(messages)
print(response.content)

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

output_parser = StrOutputParser()

chain = prompt | model | output_parser
response = chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

print(response)

In [ ]:
from openai import AzureOpenAI

client = AzureOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    api_version=os.environ["AZURE_OPENAI_API_VERSION"]
)

prompt = "Quelle est la version de la librairie OpenAI ?"

completion = client.chat.completions.create(
        model=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
        messages=[{"role": "user", "content": prompt}]
)

print(completion.choices[0].message.content.strip())

In [ ]:
# prompt content filter result in "model_extra" for azure
prompt_filter_result = completion.model_extra["prompt_filter_results"][0]["content_filter_results"]
print("\nPrompt content filter results:")
for category, details in prompt_filter_result.items():
    print(f"{category}:\n filtered={details['filtered']}\n severity={details['severity']}")

# completion content filter result
print("\nCompletion content filter results:")
completion_filter_result = completion.choices[0].model_extra["content_filter_results"]
for category, details in completion_filter_result.items():
    print(f"{category}:\n filtered={details['filtered']}\n severity={details['severity']}")

# Guardrails AI

In [ ]:
%pip install guardrails-ai

In [ ]:
%pip freeze | grep guardrails-ai

https://medium.com/@krshranjith/how-to-use-guardrail-to-langchain-c3d5312d431b

In [ ]:
import openai
import guardrails as gd
from rich import print
import pydantic

In [ ]:
rail = """
<rail version="0.1">
<output>
<object description="classified label" name="labels"></object>
<object format="valid-range: 0 1" name="scores"></object>
</output>
<prompt>

Given the following llm output about a classification label and scores, please extract a dictionary that contains the label and scores.

${llm_output}

${gr.complete_json_suffix_v2}
</prompt>
</rail>
"""

In [ ]:
import guardrails as gr

guard = gr.Guard.from_rail_string(rail)

In [ ]:
text = """
Marie, après avoir obtenu son baccalauréat au Lycée Henri-IV à Paris, a poursuivi ses études à l'Université Paris-Sorbonne où elle a obtenu une licence en lettres modernes. Passionnée par la littérature, elle a ensuite intégré Hachette Livre en tant qu'éditrice junior. Aujourd'hui, elle est rédactrice en chef dans cette même maison d'édition.

Pierre, quant à lui, a suivi une formation en ingénierie informatique à l'École Polytechnique. Après son diplôme, il a été recruté par Capgemini en tant que développeur logiciel. Grâce à son talent et à son travail acharné, il occupe désormais le poste de directeur de projet chez Dassault Systèmes.

Sophie a commencé ses études au Lycée Louis-le-Grand avant de rejoindre Sciences Po Paris. Avec un master en relations internationales en poche, elle a commencé sa carrière au sein du ministère des Affaires étrangères. Elle travaille maintenant comme diplomate à l'ambassade de France à Washington.

Enfin, Jean, après un cursus au Lycée Saint-Louis à Paris, a intégré l'ESSEC Business School. Spécialisé en finance, il a débuté chez BNP Paribas en tant qu'analyste financier. Aujourd'hui, il est directeur financier chez L'Oréal, où il supervise les investissements et la stratégie financière de la société.
"""

In [ ]:
import guardrails as gd
from pydantic import BaseModel
from typing import Optional, List
from pydantic import Field

class Person(BaseModel):
    name: str
    school: Optional[str] = Field(..., description="The school this person attended")
    company: Optional[str] = Field(..., description="The company this person works for")

class People(BaseModel):
    people: List[Person]

# class ExtractedInfo(BaseModel):
#     people: List[Person]
#     companies: List[Company]



In [ ]:
guard = gd.Guard.from_pydantic(output_class=People, prompt="Get the following objects from the text:\n\n ${text}")

In [ ]:
guard = gd.Guard.from_pydantic(output_class=People, prompt="Get the following objects from the text:\n\n Jean, après un cursus au Lycée Saint-Louis à Paris, a intégré l'ESSEC Business School. Spécialisé en finance, il a débuté chez BNP Paribas en tant qu'analyste financier. Aujourd'hui, il est directeur financier chez L'Oréal, où il supervise les investissements et la stratégie financière de la société.")

In [ ]:
import guardrails as gd
from pydantic import BaseModel
from typing import Optional, List
from pydantic import Field

class Sinistre(BaseModel):
    objet: str
    date: Optional[str] = Field(..., description="date à laquelle s'est produit le sinistre")
    degat: Optional[str] = Field(..., description="dégât provoqué par le sinistre")
    cause: Optional[str] = Field(..., description="cause des dégâts")

class Sinistres(BaseModel):
    sinistres: List[Sinistre]

In [ ]:
guard = gd.Guard.from_pydantic(output_class=People, prompt="Get the following objects from the text.")

In [ ]:
text = """
Je vous écris pour déclarer un sinistre concernant un dégât des eaux survenu dans mon domicile le 25 mai 2024. En rentrant chez moi ce soir-là, j’ai découvert que ma salle de bain était inondée. L'eau avait débordé de la baignoire en raison d’un problème de plomberie, et s'était répandue dans le couloir et la chambre adjacente.

Les dégâts sont assez importants : le parquet du couloir et de la chambre est complètement imbibé d’eau, et la peinture des murs commence déjà à cloquer. Plusieurs meubles, notamment une commode en bois et un tapis persan, ont été endommagés par l'humidité. J’ai immédiatement coupé l’eau et contacté un plombier d’urgence pour stopper la fuite.

Je vous joins à ce courrier des photos des dégâts ainsi que le devis du plombier pour la réparation de la plomberie défectueuse. Je souhaiterais savoir quelles sont les démarches à suivre pour que vous puissiez prendre en charge les frais de réparation et de remplacement des biens endommagés.
"""

In [ ]:
guard = gd.Guard.from_pydantic(output_class=Sinistres, prompt="Get the following objects from the text.")

In [ ]:
guard

In [ ]:
# raw_llm_output, validated_output, *rest = guard(
#     client.chat.completions.create(
#         model=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
#         messages=[{"role": "system", "content": "You are a helpful assistant."}]
#     ),
#     engine=os.environ.get("AZURE_OPENAI_API_ENGINE"),
#     max_tokens=1024,
#     temperature=0.3
# )

In [ ]:
text = "Jean, après un cursus au Lycée Saint-Louis à Paris, a intégré l'ESSEC Business School. Spécialisé en finance, il a débuté chez BNP Paribas en tant qu'analyste financier. Aujourd'hui, il est directeur financier chez L'Oréal, où il supervise les investissements et la stratégie financière de la société."

In [ ]:
import litellm

try:
    validated_response = guard(
        litellm.completion,
        model="azure/gpt-4o", # os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"]
        max_tokens=500,
        api_base=os.environ["AZURE_OPENAI_ENDPOINT"],
        api_version=os.environ["AZURE_OPENAI_API_VERSION"],
        api_key=os.environ["AZURE_OPENAI_API_KEY"],
        msg_history=[{"role": "user", "content": "{}".format(text)}],
        prompt_params={"text": text},
    )
except Exception as e:
    print(e)

In [ ]:
print(validated_response)

In [ ]:
print(validated_response)

LiteLLM is a lightweight wrapper that unifies the interface for over 100+ LLMs. Guardrails only supports 4 LLMs natively, but you can use Guardrails with LiteLLM to support over 100+ LLMs.

In [ ]:
!guardrails configure

In [ ]:
!guardrails hub install hub://guardrails/regex_match

In [ ]:
from guardrails import Guard, OnFailAction
from guardrails.hub import RegexMatch

guard = Guard().use(
    RegexMatch, regex="\(?\d{3}\)?-? *\d{3}-? *-?\d{4}", on_fail=OnFailAction.EXCEPTION
)

In [ ]:
guard.validate("123-456-7890")  # Guardrail passes

In [ ]:
try:
    guard.validate("1234-789-0000")  # Guardrail fails
except Exception as e:
    print(e)

In [ ]:
print(guard.history.last.tree)

In [ ]:
!guardrails hub install hub://guardrails/profanity_free

In [ ]:
# import sys


# def find_module(method_name):
#   for module in sys.modules:
#     print("checking module: ", module)
#     if hasattr(sys.modules[module], method_name):
#       return module
#     # return None

# module_name = find_module("ProfanityFree")

# print(f"The ProfanityFree method is located in the {module_name} library.")

In [ ]:
import guardrails.hub as hub


hub_init = hub.__file__

print("guardrails hub init is located at: ", hub_init)

with open(hub_init) as hub_init_file:
  print("\n")
  print(hub_init_file.read())

In [ ]:
!guardrails hub install hub://guardrails/competitor_check

In [ ]:
!guardrails hub install hub://guardrails/toxic_language

In [ ]:
import os

import litellm
from guardrails import Guard
from guardrails.hub import ProfanityFree

# litellm.set_verbose=True

# Create a Guard class
guard = Guard().use(ProfanityFree())


In [ ]:
validated_response = guard(
    litellm.completion,
    model="azure/gpt-4o", # os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"]
    max_tokens=500,
    api_base=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    msg_history=[{"role": "user", "content": "hello"}],
    # prompt_params={"text": text},
)

In [ ]:
print(validated_response)

In [ ]:
from guardrails import Guard, OnFailAction
from guardrails.hub import CompetitorCheck, ToxicLanguage

guard = Guard().use_many(
    CompetitorCheck(["X_company", "Y_company", "Z_company"], on_fail="fix"),
    ToxicLanguage(threshold=0.5, validation_method="sentence", on_fail=OnFailAction.EXCEPTION),
)

try:
    output = guard.validate(
        """La société X_company est le pire assureur du marché. A éviter !"""
    )  # Both the guardrails fail
except Exception as e:
    print(e)

In [ ]:
output

In [ ]:
try:
    validated_response = guard(
        litellm.completion,
        model="azure/gpt-4o", # os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"]
        max_tokens=500,
        api_base=os.environ["AZURE_OPENAI_ENDPOINT"],
        api_version=os.environ["AZURE_OPENAI_API_VERSION"],
        api_key=os.environ["AZURE_OPENAI_API_KEY"],
        msg_history=[{"role": "user", "content": "Quelles sont les grandes compagnies d'assurance ?"}],
    )
except Exception as e:
    print(e)

In [ ]:
print(validated_response)

In [ ]:
print(validated_response.raw_llm_output)

In [ ]:
print(validated_response.validated_output)

In [ ]:
import Levenshtein


distance = Levenshtein.distance(validated_response.raw_llm_output, validated_response.validated_output)
print(f"La distance de Levenshtein entre raw et validated est de {distance}")


In [ ]:
print(guard.history.last.tree)